## Cómo usar Vast.ai y subir archivos desde tu computadora local.


### 1. Crear y configurar una instancia en Vast.ai

#### 1.1 : Entrar a la plataforma
- Dirígete al sitio oficial: [https://vast.ai](https://vast.ai)
- Crea una cuenta si no tienes una. Inicia sesión.
- Completa el formulario de registro con tu correo electrónico y una 
contraseña segura.​
- Después de registrarte, accede a tu cuenta.​
- Dirígete a la sección de facturación para añadir créditos a tu cuenta. 
Puedes hacerlo mediante tarjeta de crédito o criptomonedas. 
El depósito mínimo es de $5 USD. ​
- Para conectarte a las instancias mediante SSH, necesitas generar una clave SSH 
y añadirla a tu cuenta de Vast.ai.
- En Windows: Abre PowerShell y genera una clave SSH ejecutando: ​ 
ssh-keygen -t rsa
- En tu cuenta de Vast.ai, ve a la sección "Account" y selecciona "Keys". 
Pega tu clave pública en el campo correspondiente y guárdala. ​

#### 1.2 En Mac o Linux:
- Abre la terminal.
- Genera una clave SSH: ssh-keygen -t rsa
- Presiona Enter para aceptar la ubicación predeterminada y deja la contraseña 
en blanco si lo deseas.
- Copia la clave pública al portapapeles:
- macOS:​ pbcopy < ~/.ssh/id_rsa.pub
- Linux:​ xclip -sel clip < ~/.ssh/id_rsa.pub
- En tu cuenta de Vast.ai, ve a la sección "Account" y selecciona "Keys". 
Pega tu clave pública en el campo correspondiente y guárdala.​

#### 1.3 : Buscar y filtrar una máquina
Haz clic en el botón **"Create"** para comenzar a buscar una instancia. 
Establece los siguientes filtros en la parte izquierda de la interfaz:

- **CUDA Version** `>= 11.8` (necesaria para modelos LLaMA recientes)
- **VRAM** `>= 22 GB` (se recomienda A5000, A6000, RTX 3090 o A100)
- **Internet Bandwidth** `>= 100 Mbps`
- **Disk Space** `>= 50 GB` (dependiendo del tamaño del dataset y los 
outputs que planeas generar)

En la opción **Image**, selecciona:
```
pytorch/pytorch:2.1.0-cuda11.8-cudnn8-runtime
```
Esta imagen ya viene con PyTorch y CUDA configurado, ideal para nuestros 
propósitos.

#### 1.4 : Habilitar puertos y opciones
Asegúrate de que estén seleccionadas las siguientes opciones:
- `Public IP` 
- `SSH Port` (para poder conectarte remotamente desde tu terminal)
- (Opcional) `Jupyter Port` si planeas abrir notebooks directamente en 
el navegador.

Haz clic en **"Rent"** para activar la máquina. Despues en la parte izquierda 
has clic en instancces y espera que termine de cargar.

---

### 2. Conexión por SSH a la instancia de Vast.ai

#### 2.1
Una vez que la instancia esté activa, la consola de Vast.ai te mostrará:
- IP pública (por ejemplo: `198.51.100.10`)
- Puerto SSH (por ejemplo: `22400`)

Abre tu terminal local (en Mac/Linux) y escribe:
```bash
ssh -p 22400 root@198.51.100.10
```
Este comando inicia sesión en la máquina remota usando el puerto correcto. 
Reemplaza `22400` y `198.51.100.10` con los valores reales que aparecen en 
tu consola de Vast.

Al conectarte, estarás en la carpeta base, normalmente `/root/` o `/workspace/` 
dependiendo de la imagen.

---

### 3. Subir archivos desde tu computadora local (Mac o Linux) al servidor de Vast.ai

#### 3.1
Para transferir archivos usaremos el comando `scp` (secure copy), 
que funciona desde cualquier terminal. o simplemente los arrastramos y esperamos
que carguen.

#### 3.2 Sintaxis general:
```bash
scp -P PUERTO archivo_local usuario@IP:/ruta/remota/destino/
```
#### Ejemplo práctico:
Supongamos que quieres subir estos dos archivos desde tu Mac:
- `extraer_mlp8_hf.py`
- `textos_oscar_1porciento.jsonl`

Y que la IP del servidor es `198.51.100.10` y el puerto SSH es `22400`. 
Entonces en tu terminal escribe:

```bash
scp -P 22400 extraer_mlp8_hf.py root@198.51.100.10:/workspace/
scp -P 22400 textos_oscar_1porciento.jsonl root@198.51.100.10:/workspace/
```

Esto hará que los archivos se copien desde tu máquina local al servidor remoto, 
específicamente a la carpeta `/workspace/`, que es donde suelen montarse 
los entornos de trabajo en esa imagen de PyTorch.

---

### 4. Verifica que los archivos llegaron correctamente

Una vez conectado por SSH, puedes verificar que los archivos fueron subidos 
correctamente con:
```bash
ls /workspace/
```
Deberías ver `extraer_mlp8_hf.py` y `textos_oscar_1porciento.jsonl` listados.





### Explicación del script `extraer_mlp8_hf.py` para ejecutarlo en Vast.ai

Aquí se  explica línea por línea el funcionamiento del script 
`extraer_mlp8_hf.py`, destinado a extraer activaciones de la capa MLP 8 del 
modelo LLaMA 3.2 y subirlas automáticamente a Hugging Face. Además, se detalla 
cómo ejecutar este script en un servidor rentado en Vast.ai.


### 1. Preparativos previos en Vast.ai

Antes de correr este script:
- Asegúrate de tener el archivo `extraer_mlp8_hf.py` y 
`textos_oscar_1porciento.jsonl` subidos en la ruta `/workspace/` de tu 
servidor.
- Estés dentro del entorno base con Python, PyTorch y Transformers instalados.

Puedes correr el script desde terminal con:
```bash
cd /workspace
python extraer_mlp8_hf.py
```

---

### 2. Explicación del script

```python
from huggingface_hub import login, HfApi
```
Importa:
- `login`: para autenticarte con Hugging Face.
- `HfApi`: para usar métodos de subida de archivos.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
```
Importa:
- `AutoTokenizer`: para convertir textos en tokens.
- `AutoModelForCausalLM`: modelo autoregresivo tipo LLaMA.

```python
import torch
import numpy as np
import os
import json
import zipfile
import shutil
```
Importa librerías necesarias para procesamiento tensorial, manejo de archivos 
y compresión ZIP.

```python
# === CONFIGURACIÓN ===
login("TU_TOKEN_AQUI")
```
Autentica el script en Hugging Face mediante tu token personal (este con
permiso write).

```python
repo_id = "naraca/mi-dataset-activaciones-llama3_2"
```
Define el repositorio donde se subirán los `.zip` generados.

```python
model_name = "meta-llama/Llama-3.2-1B"
```
Nombre del modelo base.

```python
jsonl_path = "textos_oscar_1porciento.jsonl"
```
Ruta del archivo con textos para procesar.

```python
batch_size = 32
max_length = 256
layer_idx = 8
zip_every = 1000
```
Configura:
- Tamaño del batch
- Longitud máxima de tokens
- Índice de la capa MLP a extraer
- Cada cuántos tensores se hace un `.zip`

```python
api = HfApi()
```
Instancia de `HfApi` para subir archivos a Hugging Face.

```python
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
```
Carga el tokenizador y asigna el token de padding como el token de fin de 
secuencia (para evitar errores en modelos como LLaMA).

```python
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True
)
model.eval()
```
Carga el modelo preentrenado en precisión baja (`float16`) y lo mapea 
automáticamente a CPU o GPU disponibles. Se pone en modo evaluación (`eval`).

```python
with open(jsonl_path, "r") as f:
    lines = f.readlines()
```
Carga el archivo `.jsonl` como una lista de líneas para iterarlas más adelante.

```python
mlp_outputs = []
counter = 0
```
Inicializa una lista vacía para almacenar tensores y un contador de chunks.

```python
def capture_mlp_output(module, input, output):
    mlp_outputs.append(output.detach().cpu())
```
Función `hook` que captura la salida de la capa MLP, la desconecta del grafo 
de cómputo y la envía a la RAM (CPU).

```python
mlp_module = model.model.layers[layer_idx].mlp
hook_handle = mlp_module.register_forward_hook(capture_mlp_output)
```
Registra el hook para capturar activaciones de la capa MLP 8.

```python
for i in range(0, len(lines), batch_size):
```
Iteración principal en bloques del tamaño `batch_size`.

```python
    batch_texts = [json.loads(l)["text"] for l in lines[i:i+batch_size]]
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, 
    truncation=True, max_length=max_length).to(model.device)
```
Toma un batch de textos, los tokeniza con truncamiento y padding, y los pasa 
al dispositivo del modelo.

```python
    with torch.no_grad():
        model(**inputs)
```
Ejecuta el modelo sin guardar gradientes (modo inferencia).

```python
    if len(mlp_outputs) >= zip_every:
        chunk = mlp_outputs[:zip_every]
        chunk_path = f"mlp8_chunk_{counter}.pt"
        torch.save(chunk, chunk_path)
```
Cuando se acumulan suficientes tensores (`zip_every`), se guardan en un 
archivo `.pt`.

```python
        zip_path = f"mlp8_chunk_{counter}.zip"
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            zipf.write(chunk_path)
```
Se crea un archivo `.zip` que contiene el `.pt`.

```python
        api.upload_file(
            path_or_fileobj=zip_path,
            path_in_repo=zip_path,
            repo_id=repo_id,
            repo_type="dataset"
        )
```
Sube automáticamente el archivo `.zip` al repositorio de Hugging Face indicado.

```python
        os.remove(chunk_path)
        os.remove(zip_path)
        mlp_outputs = mlp_outputs[zip_every:]
        counter += 1
```
Elimina archivos locales y actualiza el buffer y contador.

```python
hook_handle.remove()
```
Al terminar todo el proceso, se desconecta el hook para liberar recursos.



Este script permite automatizar completamente la extracción, almacenamiento y publicación de activaciones MLP8 desde un dataset de textos en un entorno GPU remoto (como Vast.ai).






## Errores comunes, recuperación y recomendaciones al ejecutar el script `extraer_mlp8_hf.py`


- Posibles errores comunes al correr el script en Vast.ai.
- Cómo recuperar el procesamiento si se interrumpe.
- Cómo verificar que los archivos subidos sean válidos.
- Recomendaciones adicionales para evitar fallos.

---

### 1. Errores comunes y cómo solucionarlos

####  `RuntimeError: CUDA out of memory`
**Causa:** El batch actual no cabe en la memoria de la GPU.  
**Solución:** Reduce el `batch_size` en el script, por ejemplo de `32` a `16` 
o `8`.

#### `Token indices sequence length is longer than the specified maximum`
**Causa:** Algún texto excede `max_length`.  
**Solución:** Asegúrate de tener `truncation=True` en el 
`tokenizer(...)`. Esto ya está hecho correctamente en el script.

####  `KeyError: 'text'`
**Causa:** El archivo `.jsonl` no tiene un campo `"text"` en alguna línea.  
**Solución:** Abre el archivo y verifica que cada línea sea un JSON válido 
con la clave `"text"`.

####  `ConnectionError` al subir a Hugging Face
**Causa:** Falla temporal de red.  
**Solución:** Asegúrate de que la instancia tiene conexión. Si es intermitente,
 puedes usar `try-except` para reintentos.

---

### 2. ¿Qué hacer si se interrumpe el script?

No necesitas volver a empezar desde cero. Para reiniciar el script de forma 
inteligente:

1. **Verifica cuántos `.zip` ya se subieron** a tu repositorio en Hugging Face.
2. **Modifica el contador manualmente** al reiniciar el script:
   ```python
   counter = N  # donde N es el siguiente número de chunk
   ```
3. **Ajusta la lista `lines = lines[M:]`** para saltarte los textos ya 
procesados.
4. **También puedes guardar un registro (`log.txt`)** con las líneas procesadas 
   para más seguridad.

---

### 3. Verificar los archivos subidos en Hugging Face

Puedes comprobar desde la interfaz de tu repositorio en:
```
https://huggingface.co/datasets/usuario/repositorio_name
```
Haz clic sobre cualquier `.zip`, descárgalo y luego localmente:

```python
import torch, zipfile
with zipfile.ZipFile("mlp8_chunk_0.zip") as zf:
    zf.extractall()
chunk = torch.load("mlp8_chunk_0.pt")
print(type(chunk), len(chunk))  # Debe ser una lista de tensores
```

---

### 4. Recomendaciones adicionales

* Usa `torch_dtype=torch.float16` para eficiencia.  
* Si la conexión de red es inestable, guarda los `.zip` y súbelos al final 
  manualmente.  
* Elimina archivos locales inmediatamente después de subirlos para ahorrar 
  espacio en disco.  
* Usa `with torch.no_grad():` para evitar gastos innecesarios de memoria.

---

Este complemento ayudará a mantener un proceso robusto, eficiente y 
recuperable si ocurre algún fallo inesperado.

